In [3]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from IPython.display import Markdown, display, update_display

In [5]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [7]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest 
pulling dde5aa3fc5ff... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a5... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   96 B                         
pulling 34bb5ab01051... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  561 B                         
verifying sha

In [97]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        try:
            self.url = url
            response = requests.get(url, headers=headers)
            self.body = response.content
            soup = BeautifulSoup(self.body, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = ""
            links = [link.get('href') for link in soup.find_all('a')]
            self.links = [link for link in links if link]
        except:
            print("website error")
    def get_contents(self):
        try:
            return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        except:
            return ""

In [11]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [13]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [15]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [49]:
ed = Website("https://huggingface.co")
ed.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ByteDance/InfiniteYou',
 '/ds4sd/SmolDocling-256M-preview',
 '/models',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Trudy/gemini-codrawing',
 '/spaces/3DAIGC/LHM',
 '/spaces/stabilityai/stable-virtual-camera',
 '/spaces/tencent/Hunyuan-T1',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/facebook/collaborative_agent_bench',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/go

In [29]:
from openai import OpenAI
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [51]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://huggingface.co - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/models
/datasets
/spaces
/posts
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/deepseek-ai/DeepSeek-V3-0324
/Qwen/Qwen2.5-Omni-7B
/manycore-research/SpatialLM-Llama-1B
/ByteDance/InfiniteYou
/ds4sd/SmolDocling-256M-preview
/models
/spaces/ByteDance/InfiniteYou-FLUX
/spaces/Trudy/gemini-codrawing
/spaces/3DAIGC/LHM
/spaces/stabilityai/stable-virtual-camera
/spaces/tencent/Hunyuan-T1
/spaces
/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1
/datasets/glaiveai/reasoning-v1-20m
/datasets/FreedomIntelligence/medical-o1-reasoning-SFT
/datasets/facebook/collaborative_agent_bench
/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pric

In [33]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [35]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ByteDance/InfiniteYou',
 '/ds4sd/SmolDocling-256M-preview',
 '/models',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Trudy/gemini-codrawing',
 '/spaces/3DAIGC/LHM',
 '/spaces/stabilityai/stable-virtual-camera',
 '/spaces/tencent/Hunyuan-T1',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/facebook/collaborative_agent_bench',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/go

In [79]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://graphql.huggingface.co/'},
  {'type': 'Company page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'Careers/Jobs page',
   'url': "https://apply.workable.com/huggingface/'"},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co/'}]}

In [81]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [89]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co'}, {'type': 'Company page', 'url': 'https://huggingface.co/brand'}, {'type': 'Allenai team member (likely CEO)', 'url': 'https://github.com/huggingface'}, {'type': 'Microsoft partnership', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Intel partnership', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Amazon partnership', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Grammarly integration', 'url': 'https://grammarly.github.io/'}, {'type': 'Writer product page', 'url': 'https://writer.huggingface.co/'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Discord server for community discussion', 'url': 'https://join.discord.huggingface.co/ '}]}
website error
website error
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Ente

In [91]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [93]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [99]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'Company Page', 'url': 'https://huggingface.co'}, {'type': 'About page', 'url': 'https://docs.huggingface.co/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn Company Page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter Company Handle', 'url': 'https://twitter.com/huggingface'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co'}, {'type': 'Discord Community', 'url': 'https://join.huggingface.codiscord'}, {'type': 'Join Careers', 'url': 'https://apply.workable.com/huggingface/'}]}
website error
website error
website error


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n1 day ago\n•\n32.5k\n•\n1.88k\nQwen/Qwen2.5-Omni-7B\nUpdated\n1 day ago\n•\n3.03k\n•\n662\nmanycore-research/SpatialLM-Llama-1B\nUpdated\n7 days ago\n•\n6.14k\n•\n754\nByteDance/InfiniteYou\nUpdated\n3 days ago\n•\n436\nds4sd/SmolDocling-256M-preview\nUpdated\n5 days ago\n•\n40.8k\n•\n994\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n473\n473\nInfiniteYo

In [101]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [103]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Pricing page', 'url': 'https://ui.endpoints.huggingface.co/pricing#endpoints'}, {'type': 'Enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co/'}, {'type': 'Documentation pages', 'url': 'https://docs.huggingface.co/'}]}
website error
website error


# Hugging Face Brochure
## About Us

Hugging Face is the AI community building the future. We're a platform where the machine learning community collaborates on models, datasets, and applications. Our mission is to make it easy for anyone to build, discover, and connect with other ML practitioners on our open-source ecosystem.

### Company Culture

At Hugging Face, we value collaboration, innovation, and openness. We believe that AI should be accessible to everyone, and we strive to create an inclusive environment where ML researchers and practitioners can share ideas, learn from each other, and accelerate their progress.

### Our Community

Our community is growing rapidly, with over 50,000 organizations using our platform. We're proud of our collaborations with leading companies like Meta, Google, Amazon, Intel, Microsoft, Grammarly, Writer, and Hunyuan T1, who are leveraging our technology to build cutting-edge AI applications.

### Products & Services

We offer a wide range of products and services, including:

* **Hugging Face Hub**: A vast catalog of pre-trained models, datasets, and applications developed by the community.
* **Spaces**: A platform for hosting, collaborating on, and deploying public models, datasets, and applications.
* **Computing**: Optimized inference endpoints for deploying models on-premise or in the cloud.

### Leadership

Our leadership team is comprised of experienced professionals with a deep understanding of AI and machine learning.

### Career Opportunities

Join our team! We're always looking for talented ML practitioners to help us build the future of AI. Check out our career page for more information.

## Model Gallery
Browse 1M+ pre-trained models and discover new applications.

*DeepSeek-V3-0324*

Qwen/Omni-7B

SpatialLM-Llama-1B

InfiniteYou FLUX

# Datasets
 Access & share datasets for any ML tasks.

*   NVidia/Llama-Nemotron-Post-Training-Dataset-v1*
*   glaive.ai/reasoning-v1-20M*

## Get Started
Accelerate your ML with our paid Compute and Enterprise solutions. Sign up today!

In [107]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [109]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co'}, {'type': 'Brand page', 'url': 'https://huggingface.com/brand/'}, {'type': 'Home (company) page', 'url': 'https://huggingface.co/company'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/', 'alt': 'Join our team'}, {'type': 'Learn page', 'url': 'https://learn.huggingface.co'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co'}, {'type': 'News/Updates page', 'url': 'https://status.huggingface.co/'}, {'type': 'Forum/Discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Twitter account', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Documentation/Transformers page', 'url': 'https://huggingface.co/docs/transformers'}, {'type': 'Documentation/Hub page', 'url': 'https://huggingface.co/docs/hu

# Hugging Face: The AI Community Building the Future

[Image of a futuristic cityscape with artificial intelligence elements, such as robots and screens displaying code]

Welcome to Hugging Face, the leading platform for the machine learning community. We are on a mission to build the future of artificial intelligence through collaboration, innovation, and open-source technology.

## About Us

Hugging Face was founded by Francis Brezillon who aims to bring together researchers, developers, and organizations from around the world to collaborate on building AI models. Our platform provides a suite of tools and resources for text, image, video, audio, and 3D models, making it easier for individuals and teams to accelerate their machine learning journey.

## Models

### Browse 1M+ Models

From deep-seek to transformers, our model repository is home to over 1 million AI models. Explore popular models like:

*   **deepseek-ai/DeepSeek-V3-0324**: An optimized version of the Deep Seek model for image generation
*   **Qwen/Qwen2.5-Omni-7B**: A novel framework for multi-modal language understanding
*   **manycore-research/SpatialLM-Llama-1B**: A spatial transformer model for natural language processing

**Select Your Model**

| Model | Description |
| --- | --- |
| InfiniteYou-FLUX  | Flexible photo recrafting while preserving your identity  |
| Gemini Co-Drawing  | Native image generation co-doodling with AI  |
| LHM   | Large animatable human model for virtual camera views |

## Datasets

With access to over 250,000 datasets, our platform enables you to train and validate your models on a diverse range of data sources.

*   **nvidia/Llama-Nemotron-Post-Training-Dataset-v1**: A dataset for learning from few-shot text classification
*   **glaiveai/reasoning-v1-20m**: An extensive benchmark for common sense reasoning

## Spaces

Our spaces feature allow users to build, share, and deploy custom models directly onto our infrastructure.

### 400k+ Applications

From chatbots to image generators, our platform hosts a vast range of applications built on top of our ecosystem.

## Community Support

Join us at [Twitter](https://twitter.com/HuggingFace) for the latest news, research opportunities, and open-source releases.